In [1]:
import time
import mujoco
import mujoco.viewer

In [2]:
n_frames = 60
height = 300
width = 480
k1, k2 = 5, 1
frames = []

In [3]:
with open('system.xml') as f:
    model = mujoco.MjModel.from_xml_string(f.read())
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model, height, width)
mujoco.mj_resetData(model, data)
mujoco.mj_resetDataKeyframe(model, data, 0)

with mujoco.viewer.launch_passive(model, data) as viewer:
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  integral = 0
  while viewer.is_running() and time.time() - start < 300:
    step_start = time.time()
    mujoco.mj_step(model, data)
    x_current = data.sensor('sensor').data.copy()[0]
    integral += x_current * model.opt.timestep
    cntrl= -k1 * x_current -k2 * integral
    data.ctrl[0] = cntrl
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time % 2)
    viewer.sync()
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)